In [81]:
import cx_Oracle
import pandas as pd
import requests
#from xml.etree.ElementTree import parse
from xml.etree.ElementTree import fromstring

In [82]:
access_key = 'xMa0cbwrTEM1UQzMljSLPYi29E5u1BLZysXwaxEgC6n44Ymoi/V86yK/giMFMiKE8zuvzHcSYFHFKOwwUy8dtw=='

TOTAL_NUM = 650
MAX_TRANSACTION = 30

In [83]:
con = cx_Oracle.connect('open_source/1111@localhost:1521/xe')
cur = con.cursor()

In [84]:
def xml_to_oracle(data_root,data_root_name):
    for row in data_root.iter(data_root_name):
        sql_insert = """
            insert into TSUNAMI_EV_SHELTER(일련번호,시도명,시군구명,대피지구명,대피장소명,주소,수용가능인원수,대피소분류명,
            해변으로부터거리,내진적용여부,해발높이,경도,위도)
            values(:일련번호,:시도명,:시군구명,:대피지구명,:대피장소명,:주소,:수용가능인원수,:대피소분류명,
            :해변으로부터거리,:내진적용여부,:해발높이,:경도,:위도)
        """
#         rename_dict = {'id':'일련번호', 'sido_name':'시도명', 'sigungu_name':'시군구명', 'remarks':'대피지구명',
#                'shel_nm':'대피장소명', 'address':'주소', 'lon':'경도', 'lat':'위도', 'shel_av':'수용가능인원수',
#                'lenth':'해변으로부터거리', 'shel_div_type':'대피소 분류명', 'seismic':'내진적용여부', 'height':'해발높이'}

        일련번호 = row.find('id').text
        시도명 = row.find('sido_name').text
        시군구명 = row.find('sigungu_name').text
        대피지구명 = row.find('remarks').text
        대피장소명 = row.find('shel_nm').text
        주소 = row.find('address').text
        수용가능인원수 = row.find('shel_av').text
        대피소분류명 = row.find('shel_div_type').text
        해변으로부터거리 = row.find('lenth').text
        내진적용여부 = row.find('seismic').text
        해발높이 = row.find('height').text
        경도 = row.find('lon').text
        위도 = row.find('lat').text
        
        cur.execute(sql_insert,(일련번호,시도명,시군구명,대피지구명,대피장소명,주소,수용가능인원수,
                                대피소분류명,해변으로부터거리,내진적용여부,해발높이,경도,위도))
        con.commit()

In [85]:
def get_request_url(page_no):
    url = 'http://apis.data.go.kr/1741000/TsunamiShelter3/getTsunamiShelter1List'
    params = {'serviceKey' : access_key, 'numOfRows' : MAX_TRANSACTION, 'pageNo' : page_no}
    #params = {'serviceKey' : access_key}
    TsunamiShelter = requests.get(url, params = params)

    return TsunamiShelter.text

In [86]:
# raw_xml = get_request_url()

In [87]:
# root = fromstring(raw_xml)

In [88]:
# tree = parse('지진해일_긴급대피장소.xml')
# data_root = tree.getroot()

In [89]:
# xml_to_oracle(data_root,'row')

In [90]:
request_total_num = ( TOTAL_NUM // MAX_TRANSACTION ) + 1

rename_dict = {'id':'일련번호', 'sido_name':'시도명', 'sigungu_name':'시군구명', 'remarks':'대피지구명',
               'shel_nm':'대피장소명', 'address':'주소', 'lon':'경도', 'lat':'위도', 'shel_av':'수용가능인원수',
               'lenth':'해변으로부터거리', 'shel_div_type':'대피소 분류명', 'seismic':'내진적용여부', 'height':'해발높이'}

REDEFINE_COLUMNS = ['일련번호','시도명','시군구명','대피지구명','대피장소명','주소','수용가능인원수',
                    '대피소 분류명','해변으로부터거리','내진적용여부','해발높이','경도','위도']

In [91]:
for page_no in range(1, request_total_num + 1):
    raw_xml = get_request_url(page_no)
    root = fromstring(raw_xml)

    #tree = parse("지진해일_긴급대피장소.xml")
    data_root = root
    
    xml_to_oracle(data_root,'row')